# HADOOP-16435: RpcMetrics should not be retained forever

[Link to Jira report.](https://issues.apache.org/jira/browse/HADOOP-16435)

Package created by Max Goetzmann.

## Reserve bare metal and floating IP

In [9]:
import chi
from chi.lease import lease_duration, add_node_reservation, create_lease, add_fip_reservation

SITE = 'CHI@NU'
CHARGE_CODE = 'CHI-220969'
RESERVATION_DAY_LENGTH = 1
LEASE_NAME = 'bug-repro-HADOOP-16435'

chi.use_site(SITE)
chi.set('project_name', CHARGE_CODE)

node_type = "compute_skylake"
start_date, end_date = lease_duration(days=RESERVATION_DAY_LENGTH)

# Build list of reservations (in this case there is only one reservation)
reservations = []
add_node_reservation(reservations, count=1, node_type=node_type)
add_fip_reservation(reservations, count=1)

# Create the lease
lease = create_lease(LEASE_NAME, reservations, start_date=start_date,
                     end_date=end_date)


Now using CHI@NU:
URL: https://sl-ciab.northwestern.edu
Location: Evanston, Illinois, USA
Support contact: help@chameleoncloud.org


## Launch bare metal

In [ ]:
from chi.lease import get_node_reservation
from chi.server import create_server, associate_floating_ip, wait_for_tcp

SERVER_NAME = LEASE_NAME
reservation_id = get_node_reservation(LEASE_NAME)
server = create_server(SERVER_NAME, 
                       image_name="CC-Ubuntu22.04",
                       reservation_id=reservation_id)

server_id = server.id
ip = associate_floating_ip(server_id)

# Wait for SSH connectivity over port 22
wait_for_tcp(ip, port=22)

## Upload and execute setup script 

In [13]:
import os
from chi import ssh

def list_files(dir):
    file_list = []
    for root, _, files in os.walk(dir):
        for file in files:
            file_list.append(dir + os.path.relpath(os.path.join(root, file), dir))
    return file_list

bin_files = list_files('./bin/')

with ssh.Remote(ip) as conn:
    # Upload setup files 
    conn.run(f'mkdir {LEASE_NAME}')
    conn.run(f'cd {LEASE_NAME}')
    for file in bin_files:
        conn.put(file)
    
    # Execute setup
    conn.run("prepare.sh")



['./bin/setup.sh', './bin/protobuf_install.sh']


## Perform experiment

## Process and output experiment results

In [ ]:
# Filler